In [1]:
import pandas as pd
import numpy as np
import duckdb
import warnings
import os

import xgboost as xgb
from sklearn.preprocessing import LabelEncoder

# email
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart

import warnings
from datetime import datetime, timedelta

pd.set_option('display.max_columns', None)
warnings.filterwarnings("ignore")

categories = ['PTS', 'AST', 'REB', 'PR', 'PA', 'RA', 'PRA', 'TPM', 'STL', 'BLK', 'STL_BLK']
con = duckdb.connect(database=":memory:")

cwd = os.path.abspath(os.getcwd()).replace("\\", "/")
if cwd.startswith("C:/Users/Rodolfo/"):
    RUN_LOCATION = "local"
else:
    RUN_LOCATION = "cloud"
time_offset = {"local": 3, "cloud": -5}
now = str((datetime.now() + timedelta(hours=time_offset[RUN_LOCATION]) + timedelta(hours=-3)).date())
print(f"Today's date:", now)

Today's date: 2026-01-09


In [2]:
%run ./common_utils.ipynb

# Initial Functions

In [3]:
def email(model, error):
    
    # Email details
    sender_email = "rodolfoe7157@gmail.com"
    receiver_email = "rodolfoe7157@gmail.com"
    password = "cqgu bfey cnyx sfue"  # See note below

    subject = "NBA create_Predictions error"
    body = f"Model: {model}_model\nERROR: {error}"

    # Create message
    msg = MIMEMultipart()
    msg['From'] = sender_email
    msg['To'] = receiver_email
    msg['Subject'] = subject
    msg.attach(MIMEText(body, 'plain'))

    # Connect to Gmail SMTP server and send
    with smtplib.SMTP_SSL('smtp.gmail.com', 465) as server:
        server.login(sender_email, password)
        server.send_message(msg)

    print("Email sent successfully!")

### Create missing_df

In [4]:
def load_df(file_name):
    df = pd.DataFrame()
    for i in [2021, 2022, 2023, 2024, 2025]:
        df_temp = pd.read_csv(f"../tables/{i}/{file_name}.csv")
        df_temp['Season'] = i
        df = pd.concat([df, df_temp])
        
    if 'Date' in df.columns:
        df['Date'] = pd.to_datetime(df.Date)
    if file_name == "season_gamelogs":
        df = df[~df[['Date', 'Team', 'Player']].duplicated(keep='last')]
    
    return df

In [20]:
def create_base_df():
    
    # Load dfs
    df = load_df('parlay_stats')
    df2 = load_df('nba_schedule')
    df3 = load_df('season_gamelogs')
    df4 = load_df('injuries')
    df5 = load_df('plyr_pos_xref')
    df6 = load_df('daily_lineups')
    gmlog_cols = ['game_id', 'Player', 'MP', 'PF']
    df7 = load_df('h1_season_gamelogs')[gmlog_cols].rename(columns={"MP": "MP_h1", "PF": "PF_h1"})
    df8 = load_df('h2_season_gamelogs')[gmlog_cols].rename(columns={"MP": "MP_h2", "PF": "PF_h2"})
    df9 = load_df('q1_season_gamelogs')[gmlog_cols].rename(columns={"MP": "MP_q1", "PF": "PF_q1"})
    df10 = load_df('q2_season_gamelogs')[gmlog_cols].rename(columns={"MP": "MP_q2", "PF": "PF_q2"})
    df11 = load_df('q3_season_gamelogs')[gmlog_cols].rename(columns={"MP": "MP_q3", "PF": "PF_q3"})
    df12 = load_df('q4_season_gamelogs')[gmlog_cols].rename(columns={"MP": "MP_q4", "PF": "PF_q4"})

    df3 = df3.rename(columns={"3PM": "TPM", "3PA": "TPA", "3P%": "TP%", "TRB": "REB"}).drop(['Pos', 'Opp'], axis=1)
    df3['PR'] = df3.PTS + df3.REB 
    df3['PA'] = df3.PTS + df3.AST
    df3['RA'] = df3.REB + df3.AST
    df3['PRA'] = df3.PTS + df3.REB + df3.AST
    df3['STL_BLK'] = df3.STL + df3.BLK
    df = df.merge(df3, on=['Season', 'Date', 'Team', 'Player'], how='left')

    df_mtch = df2[['Season', 'Date', 'AwayABV', 'HomeABV', 'AwayPTS', 'HomePTS', 'AwayB2B', 'HomeB2B', 'is_OT', 'cup_gm', 'pstszn_gm']]
    df_mtch['Team_type'] = 'Away'
    df_mtch = df_mtch.rename(columns={"AwayABV": "Team", "HomeABV": "Opp", "AwayB2B": "B2B"})[['Season', 'Date', 'Team', 'AwayPTS', 'HomePTS', 'Opp', 'B2B', 'is_OT', 'cup_gm', 'pstszn_gm', 'Team_type']]
    df_mtch2 = df_mtch.copy().rename(columns={"Team": "Opp", "Opp": "Team", "HomeB2B": "B2B"})[['Season', 'Date', 'Team', 'AwayPTS', 'HomePTS', 'Opp', 'B2B', 'is_OT', 'cup_gm', 'pstszn_gm']]
    df_mtch2['Team_type'] = 'Home'
    df_mtch = pd.concat([df_mtch, df_mtch2])
    df_mtch = df_mtch[['Season', 'Date', 'Team', 'Team_type', 'AwayPTS', 'HomePTS', 'is_OT', 'cup_gm', 'pstszn_gm']]
    df_mtch = df_mtch.sort_values(["Team", "Date"])
    df_mtch['team_game_num'] = df_mtch.groupby(["Team", "Season"]).cumcount() + 1
    df_mtch['Spread'] = np.where(df_mtch.Team_type == 'Home', df_mtch.HomePTS - df_mtch.AwayPTS, df_mtch.AwayPTS - df_mtch.HomePTS)
    df_mtch['Total'] = df_mtch.AwayPTS + df_mtch.HomePTS
    df_mtch['is_Win'] = np.where(df_mtch.Spread > 0, 1, 0)
    df_mtch['Szn_Wins'] = df_mtch.groupby(['Season', 'Team'])['is_Win'].cumsum()
    df = df.drop(['Season', 'Team_type'], axis=1).merge(df_mtch, on=['Date', 'Team'])

    df = df.merge(df4[['Date', 'Team', 'Player', 'Status']], on=['Date', 'Team', 'Player'], how='left')
    df['Status'] = np.where((df.Active == 1) & (df.Status.isnull()), 'Available', df.Status)
    df['Status'] = np.where((df.Active == 0), 'Out', df.Status)
    df['Status'] = np.where((df.Status == 'Out') & (df.Active != 0), 'Available', df.Status)

    df6['role'] = 1
    df = df.merge(df6.drop('Pos', axis=1), on=['Season', 'Date', 'Team', 'Player'], how='left')
    df['role'] = df.role.fillna(2).astype(int)
    df['role'] = np.where(((df.MP < 8) & (df.role == 2)), 3, df.role)

    # Add gmlog splits
    df_gmlog_comb = df7.merge(df8, on=['game_id', 'Player'])
    for df_loop in (df9, df10, df11, df12):
        df_gmlog_comb = df_gmlog_comb.merge(df_loop, on=['game_id', 'Player'])
    df = df.merge(df_gmlog_comb, on=['game_id', 'Player'], how='left')
    
    global team_encoder, player_encoder, team_type_encoder, position_encoder, status_encoder
    team_encoder = LabelEncoder()
    player_encoder = LabelEncoder()
    team_type_encoder = LabelEncoder()
    position_encoder = LabelEncoder()
    status_encoder = LabelEncoder()

    # Encode string cols
    team_encoder.fit(pd.concat([df["Team"], df["Opp"]], axis=0))
    players_fits = pd.concat([df["Player"], df3["Player"]], axis=0)
    players_fits = pd.concat([players_fits, df4["Player"]], axis=0).drop_duplicates()
    player_encoder.fit(players_fits)
    df["Team"] = team_encoder.transform(df["Team"])
    df["Opp"] = team_encoder.transform(df["Opp"])
    df["Player_name"] = df.Player
    df["Player"] = player_encoder.transform(df["Player"])
    df["Pos"] = position_encoder.fit_transform(df["Pos"])
    df['Team_type'] = team_type_encoder.fit_transform(df['Team_type'])
    df["Status"] = status_encoder.fit_transform(df["Status"])

    return df

In [21]:
def create_df_missing(df, pred_col):
    
    df3 = load_df('season_gamelogs')
    df3 = df3.rename(columns={"3PM": "TPM", "3PA": "TPA", "3P%": "TP%", "TRB": "REB"}).drop(['Pos', 'Opp'], axis=1)
    df3['PR'] = df3.PTS + df3.REB 
    df3['PA'] = df3.PTS + df3.AST
    df3['RA'] = df3.REB + df3.AST
    df3['PRA'] = df3.PTS + df3.REB + df3.AST
    df3['STL_BLK'] = df3.STL + df3.BLK
    df4 = load_df('injuries')
    # Fill missing games from injuries.csv
    df_pred = create_base_df()
    team_games = df_pred[['Season', 'Team', 'Date']].drop_duplicates()
    players = df_pred[['Season','Player','Team']].drop_duplicates()
    fabricated = (players.sort_values('Season').groupby('Player', as_index=False).last())
    fabricated['Season'] = fabricated['Season'] + 1
    players = pd.concat([players, fabricated], ignore_index=True).drop_duplicates(['Season','Player','Team'])
    expanded = team_games.merge(players, on=['Season', 'Team'], how='left')

    df3["Team"] = team_encoder.transform(df3["Team"])
    df3["Player"] = player_encoder.transform(df3["Player"])
    df4["Team"] = team_encoder.transform(df4["Team"])
    df4["Player"] = player_encoder.transform(df4["Player"])
    df5 = load_df('plyr_pos_xref')
    df5['Team'] = team_encoder.transform(df5["Team"])
    df5['Player'] = player_encoder.transform(df5["Player"])

    expanded = expanded.merge(df3[['Season', 'Player', 'Date', 'MP']], on=['Season', 'Player', 'Date'], how='left').drop_duplicates(['Season', 'Date', 'Player', 'Team'])
    expanded = expanded[(expanded.MP.isnull()) & (expanded.Date != now)].drop('MP', axis=1)
    expanded = pd.concat([expanded, df4[df4.Status == 'Out'][['Season', 'Team', 'Date', 'Player']]])
    df4 = df4.merge(expanded, on=['Season', 'Date', 'Team', 'Player'], how='right')

    # Grab outs from players season gamelogs
    df4 = df4.merge(df3, on=['Season', 'Date', 'Team', 'Player'], how='outer')
    df4['Status'] = np.where(((df4.Active == 1) | (df4.MP > 0)), 'Available', df4.Status)
    df4['Status'] = np.where(((df4.Active == 0) | (df4.MP == 0) | (df4.MP.isnull())), 'Out', df4.Status)
    df4['Status'] = np.where((df4.Status == 'Out') & (df4.MP > 0), 'Available', df4.Status)
    df4['Status'] = np.where((df4.Status != 'Out') & (df4.MP == 0), 'Out', df4.Status)
    df4 = df4[df4.Status == 'Out'][['Season', 'Date', 'Team', 'Player']].drop_duplicates()
    
    df_missing = df[['Season', 'Date', 'Team', 'Player', 'role', pred_col]].copy()
    df_missing[f'{pred_col}_L10'] = (
        df_missing.groupby(['Player','Season'])[pred_col]
                  .transform(lambda x: x.rolling(10, min_periods=1).mean())
    )
    df_missing = pd.merge_asof(df4, df_missing[["Season", "Player", "Date", "role", f"{pred_col}_L10"]], 
                      on="Date", by=["Player", "Season"], direction="backward", allow_exact_matches=True).dropna()   
    df_missing = df_missing.merge(df5, on=['Season', 'Team', 'Player'])

    out_minutes = (
    df_missing
      .groupby(["Season", "Date", "Team"])
      .agg(
#           out_minutes_avg=("MP_L10", "mean"),
#           team_mins_available=("MP_L10", lambda x: x.sum()),
#           plyr_mins_available=("MP_L10", lambda x: (x.sum())/5),    
          starters_out=("role", lambda x: (x == 1).sum())
      )
      .reset_index()
    )

#     out_minutes["star_weighted_out"] = (
#         df_missing.assign(weight=lambda x: x.MP_L10 / x.MP_L10.max())
#               .groupby(["Season", "Date", "Team"])
#               .apply(lambda g: (g.MP_L10 * g.weight).mean())
#               .values
#     )    
    
    return out_minutes

# x = create_df_missing(df_pred, 'MP')
# display(x[(x.Season == 2025) & (x.Team == 11) & (x.Date == now)])

# Minutes Projection Model

In [22]:
def setup_df_mins(con, df):

    df = df[['Season', 'Date', 'Team', 'Team_type', 'Opp', 'Player', 'Pos', 'role', 'B2B', 
             'MP', 'MP_h1', 'MP_h2', 'MP_q1', 'MP_q2', 'MP_q3', 'MP_q4', 
             'PF', 'PF_h1', 'PF_h2', 'PF_q1', 'PF_q2', 'PF_q3', 'PF_q4', 
             'Spread', 'team_game_num', 'pstszn_gm', 'is_OT']]
    
    for col in ['MP']:
        for N in [3, 5, 10]:
            df[f'{col}_L{N}_avg'] = (
                df.groupby(['Player', 'Season'])[col]
                  .rolling(window=N, min_periods=1)
                  .mean()
                  .shift(1)
                  .reset_index(level=[0, 1], drop=True)
            )
            df[f'prev_team_mins_pct_L{N}'] = df[f'{col}_L{N}_avg'] / 240

    games_last_7_days = df.sort_values(['Player', 'Season', 'Date']).groupby(['Player', 'Season']).rolling('7D', on='Date', closed='left')['MP'].count().reset_index().rename(columns={"MP": "gms_L7_days"})
    games_last_7_days = games_last_7_days.drop_duplicates(
        subset=['Player', 'Season', 'Date']
    )
    df = df.merge(games_last_7_days, on=['Player', 'Season', 'Date'])
    df['gms_L7_days'] = df.gms_L7_days.fillna(0).astype(int)
        
    for N in [1, 3, 5]:
        df[f"recent_role_L{N}"] = (
            df
            .groupby(["Player", "Season"])["role"]
            .rolling(5, min_periods=1)
            .apply(lambda arr: np.bincount(arr.astype(int), minlength=4).argmax(), raw=True)
            .reset_index(level=[0, 1], drop=True)
        )
    df['missed_games'] = (df.groupby(['Player', 'Team', 'Season'])['team_game_num'].diff().sub(1).fillna(0).astype(int))
    df["DaysLstGm"] = (df.groupby("Player")["Date"].diff().dt.days).fillna(0).astype(int)
    
    df['game_spread_type'] = 0
    df['game_spread_type'] = np.where(abs(df.Spread) < 4, 1, df.game_spread_type) 
    df['game_spread_type'] = np.where((abs(df.Spread) >= 4) & (abs(df.Spread) <= 10), 2, df.game_spread_type) 
    df['game_spread_type'] = np.where((abs(df.Spread) >= 11) & (abs(df.Spread) <= 15), 3, df.game_spread_type) 
    df['game_spread_type'] = np.where(abs(df.Spread) > 16, 4, df.game_spread_type) 

    # Tell model games exist after players injuries/susp
    team_games = df[['Season', 'Team', 'Date', 'team_game_num']].drop_duplicates()
    players = df[['Season','Player','Team']].drop_duplicates()
    fabricated = (players.sort_values('Season').groupby('Player', as_index=False).last())
    fabricated['Season'] = fabricated['Season'] + 1
    players = pd.concat([players, fabricated], ignore_index=True).drop_duplicates(['Season','Player','Team'])
    expanded = team_games.merge(players, on=['Season', 'Team'], how='left')
    expanded = expanded.merge(df[['Season', 'Player', 'Date', 'MP']], on=['Season', 'Player', 'Date'], how='left').drop_duplicates(['Season', 'Date', 'Player', 'Team'])
    expanded['player_played'] = expanded['MP'].notna().astype(int)
    expanded['team_played_no_player'] = ((expanded['player_played'] == 0)).astype(int)
    expanded['tm_plays_after'] = (expanded.groupby(['Player'])['team_played_no_player'].shift(-1))
    expanded['missed_gms_aftr'] = 0
    expanded['missed_gms_aftr'] = np.where((expanded.player_played == 1) & (expanded.tm_plays_after == 1), 1, expanded.missed_gms_aftr)
    df = df.merge(expanded[['Date', 'Team', 'Player', 'missed_gms_aftr']], on=['Date', 'Team', 'Player'])
    
    df2 = create_df_missing(df, 'MP')
    df = df.merge(df2, on=["Season", "Date", "Team"], how='left')
#     df['starters_out'] = df.starters_out.fillna(0)

    df['MP_change_pct_L10'] = (df['MP'] - df['MP_L10_avg']) / df['MP_L10_avg']
    MP_Inc_conds = (
                    ((df.role == 1) & (df['MP_change_pct_L10'] >= 0.05)) | 
                    ((df.role == 2) & (df['MP_change_pct_L10'] >= 0.10)) | 
                    ((df.role == 3) & (df['MP_change_pct_L10'] >= 0.15) & (df['MP_L10_avg'] > 5))
                   )
    df['MP_increase'] = np.where(MP_Inc_conds, 1, 0)
    Early_stop_conds = (
        ((df.role == 1) & (df['MP_change_pct_L10'] <= -0.35)) | 
        ((df.role == 2) & (df['MP_change_pct_L10'] <= -0.45)) | 
        ((df.role == 3) & (df['MP_change_pct_L10'] <= -0.55)) | 
        ((df.role != 3) & (df['MP'] < 8))
    )
    df['Early_stop'] = np.where(Early_stop_conds, 1, 0)
    df['Early_Exit'] = np.where(((df.MP_q2 + df.MP_h2)== 0 & (df.role == 1)), 1, 0)
    
    Foul_Trouble_cond = (
        (df.PF_q1 >= 2) |
        (df.PF_h1 >= 3) |
        ((df.PF_h1 + df.PF_q3) >= 4) |
        (df.PF_q4 >= 5)
    )
    df['Foul_Trouble'] = np.where(Foul_Trouble_cond, 1, 0)
    
    df['Injured'] = ((df.Early_stop == 1) & (df.missed_gms_aftr > 0)).astype(int)
    df['return_game'] = ((df.groupby('Player')['Injured'].shift(1) == 1) & (df.missed_games > 0)).astype(int)
    df['games_since_return'] = (df.groupby('Player')['return_game'].cumsum())
    df['games_since_return'] = (df.groupby(['Player', 'games_since_return']).cumcount())
    df['ramp_phase'] = 0
    df.loc[df.games_since_return == 0, 'ramp_phase'] = 1
    df.loc[df.games_since_return.isin([1, 2, 3]), 'ramp_phase'] = 2

    df = df.drop(['Season', 'Team_type', 'MP_h1', 'MP_h2', 'MP_q1', 'MP_q2', 'MP_q3', 'MP_q4',
                  'PF', 'PF_h1', 'PF_h2', 'PF_q1', 'PF_q2', 'PF_q3', 'PF_q4', 
                  'missed_gms_aftr', 'games_since_return', 'return_game', 'MP_change_pct_L10'], axis=1)      
    return df

# Main Model

In [31]:
def setup_df_main(df, tgt_stat):
    
    # Stat dependent features 
    if tgt_stat == 'PTS':
        tgt_stat_cols = ['TPM', 'FG', 'FT', 'TPA', 'FGA', 'FTA']
        df = df[['Season', 'Date', 'Team', 'Opp', 'Player', 'Pos', 'MP', 'team_game_num', 
         'PTS', 'TPM', 'FG', 'FGA', 'TPA', 'FT', 'FTA', 
         f'Off_{tgt_stat}', f'Off_L3_{tgt_stat}', f'Off_L5_{tgt_stat}', f'Off_L10_{tgt_stat}', f'Off_{tgt_stat}_Rk',
         f'Def_{tgt_stat}', f'Def_L3_{tgt_stat}', f'Def_L5_{tgt_stat}', f'Def_L10_{tgt_stat}', f'Def_{tgt_stat}_Rk',
         'Spread', 'Total', 'is_OT']]
        
        # Efficiency metrics
        df['three_rate_raw'] =  np.where(df.FGA > 0, df['TPA'] / df['FGA'], 0)
        df['ft_rate_raw']    =  np.where(df.FGA > 0, df['FTA'] / df['FGA'], 0)
        df['eFG_raw'] = (df['FG'] + 0.5 * df['TPM']) / df['FGA']
        df['TS_raw'] = df['PTS'] / (2 * (df['FGA'] + 0.44 * df['FTA']))    
        df['usage_proxy_raw'] =  np.where(df.MP > 0, (df['FGA'] + 0.44 * df['FTA']) / df['MP'], 0)
        
        for w in [3, 5, 10]:
            for metric in ['three_rate', 'ft_rate', 'eFG', 'TS', 'usage_proxy']:
                col = f"{metric}_L{w}"
                df[col] = (
                    df.groupby(['Player','Season'])[f'{metric}_raw']
                      .rolling(w, min_periods=1)
                      .mean()
                      .shift(1)
                      .reset_index(level=[0,1], drop=True)
                )
        for metric in ['three_rate', 'ft_rate', 'eFG', 'TS', 'usage_proxy']:
            col = f'{metric}_weighted'
            df[col] = (
                0.6 * df[f'{metric}_L3'] +
                0.3 * df[f'{metric}_L5'] +
                0.1 * df[f'{metric}_L10']
            )
            df = df.drop(f'{metric}_raw', axis=1)
        
    elif tgt_stat == 'PRA':
        tgt_stat_cols = ['PTS', 'REB', 'AST', 'TPM', 'FG']
        df = df[['Season', 'Date', 'Team', 'Opp', 'Player', 'Pos', 'MP', 'team_game_num', 
         'PTS', 'AST', 'REB', 'PR', 'PA', 'RA', 'PRA', 'TPM', 'STL', 'BLK', 'STL_BLK', 
         'FG', 'FGA', 'TPA', 'FT', 'FTA', 
         f'Off_{tgt_stat}', f'Off_L3_{tgt_stat}', f'Off_L5_{tgt_stat}', f'Off_L10_{tgt_stat}', f'Off_{tgt_stat}_Rk',
         f'Def_{tgt_stat}', f'Def_L3_{tgt_stat}', f'Def_L5_{tgt_stat}', f'Def_L10_{tgt_stat}', f'Def_{tgt_stat}_Rk',
         'Spread', 'Total', 'is_OT']]
        
        df['usage_proxy_raw'] =  np.where(df.MP > 0, (df['FGA'] + 0.44 * df['FTA']) / df['MP'], 0)
        for w in [3, 5, 10]:
            df[f"usage_proxy_L{w}"] = (
                df.groupby(['Player','Season'])[f'usage_proxy_raw']
                  .rolling(w, min_periods=1)
                  .mean()
                  .shift(1)
                  .reset_index(level=[0,1], drop=True)
            )
        df['usage_proxy_weighted'] = (
            0.6 * df[f'usage_proxy_L3'] +
            0.3 * df[f'usage_proxy_L5'] +
            0.1 * df[f'usage_proxy_L10']
        )
        df = df.drop('usage_proxy_raw', axis=1)
        
        
    else:
        tgt_stat_cols = []
        df = df[['Season', 'Date', 'Team', 'Opp', 'Player', 'Pos', 'MP', 'team_game_num', 
         'PTS', 'AST', 'REB', 'PR', 'PA', 'RA', 'PRA', 'TPM', 'STL', 'BLK', 'STL_BLK',
         'FG', 'FGA', 'TPA', 'FT', 'FTA', 
          f'Off_{tgt_stat}', f'Off_L3_{tgt_stat}', f'Off_L5_{tgt_stat}', f'Off_L10_{tgt_stat}', f'Off_{tgt_stat}_Rk',
          f'Def_{tgt_stat}', f'Def_L3_{tgt_stat}', f'Def_L5_{tgt_stat}', f'Def_L10_{tgt_stat}', f'Def_{tgt_stat}_Rk',
         'Spread', 'Total', 'is_OT']]

    
    # Create rolling + lag features    
    for col in ['MP'] + tgt_stat_cols:
        df[f'{col}_lst_gm'] = (
            df
            .groupby(['Player', 'Season'])[col]
            .shift(1)
        )
        for N in [1, 3, 5, 10]:
            df[f'{col}_L{N}_avg'] = (
                df.groupby(['Player', 'Season'])[col]
                  .rolling(window=N, min_periods=1)
                  .mean()
                  .shift(1)
                  .reset_index(level=[0, 1], drop=True)
            )

    # Role identifiers features
    df['reserve_td'] = (df.MP < 8).astype(int)
    df['bench_td']   = ((df.MP >= 8) & (df.MP <= 25)).astype(int)
    df['starter_td'] = (df.MP > 25).astype(int)
    role_counts = df.groupby(['Season', 'Player'])[['reserve_td', 'bench_td', 'starter_td']].sum()
    role_counts['most_common_role'] = role_counts[['reserve_td', 'bench_td', 'starter_td']].idxmax(axis=1)
    role_counts['reserve'] = (role_counts['most_common_role'] == 'reserve_td').astype(int)
    role_counts['bench']   = (role_counts['most_common_role'] == 'bench_td').astype(int)
    role_counts['starter'] = (role_counts['most_common_role'] == 'starter_td').astype(int)
    df = df.merge(role_counts[['reserve', 'bench', 'starter']], on=['Season', 'Player'], how='left')
    df['role'] = 0
    df['role'] = np.where(df.starter == 1, 1, df.role)
    df['role'] = np.where(df.bench == 1, 2, df.role)
    df['role'] = np.where(df.reserve == 1, 3, df.role)
    
    for N in [1, 3, 5]:
        for role in ['reserve_td', 'bench_td', 'starter_td']:
            df[f'{role}_last{N}'] = (
                df.sort_values(['Player', 'Date']).groupby('Player')[role]
                  .rolling(N, min_periods=1)
                  .sum()
                  .shift(1)
                  .reset_index(0, drop=True)
            )
        rec_role_cols = [f'{role}_last{N}' for role in ['reserve_td', 'bench_td', 'starter_td']]
        df[f'recent_most_common_role_L{N}'] = df[rec_role_cols].idxmax(axis=1)
        df[f'recent_role_L{N}'] = 0
        df[f'recent_role_L{N}'] = np.where(df[f'recent_most_common_role_L{N}'] == f'starter_td_last{N}', 1, df[f'recent_role_L{N}'])
        df[f'recent_role_L{N}'] = np.where(df[f'recent_most_common_role_L{N}'] == f'bench_td_last{N}', 2, df[f'recent_role_L{N}'])
        df[f'recent_role_L{N}'] = np.where(df[f'recent_most_common_role_L{N}'] == f'reserve_td_last{N}', 3, df[f'recent_role_L{N}'])      
        df = df.drop(f'recent_most_common_role_L{N}', axis=1)
        for role in ['reserve_td', 'bench_td', 'starter_td']:
            df = df.drop(f'{role}_last{N}', axis=1)
    
    df['game_spread_type'] = 0
    df['game_spread_type'] = np.where(abs(df.Spread < 6), 1, df.game_spread_type) 
    df['game_spread_type'] = np.where((abs(df.Spread >= 6) & abs(df.Spread <= 14)), 2, df.game_spread_type) 
    df['game_spread_type'] = np.where(abs(df.Spread > 14), 3, df.game_spread_type) 
    
    for col in categories + ['Season', 'FG', 'FGA', 'FT', 'FTA', 'TPM', 'TPA', 
                             'reserve_td', 'reserve', 'bench_td', 'bench', 'starter_td', 'starter'] + tgt_stat_cols:
        if col == tgt_stat:
            continue
        if col in df.columns:
            df = df.drop(col, axis=1)
        
    return df

### Today's predictions

In [35]:
def generate_predictions(tgt_stat):
    
    df_pred = create_base_df()
        
    mins_model = xgb.XGBRegressor()
    mins_model.load_model("../ML_models/mins_model.json")
    stat_model = xgb.XGBRegressor()
    stat_model.load_model(f"../ML_models/{tgt_stat}_model.json")
    
    df_lines = pd.read_csv(f"../tables/2025/parlay_lines.csv")
    df_lines['Date'] = pd.to_datetime(df_lines.Date)
    df_lines = df_lines[~(df_lines.Team.isnull())]

    # Predict Mins
    df_lines["Team"] = team_encoder.transform(df_lines["Team"])
    df_pred = df_pred.merge(df_lines[['Date', 'Team', 'Spread', 'Total']], on=['Date', 'Team'], how='left')
    df_pred = df_pred[~df_pred[['Date', 'Team', 'Player']].duplicated(keep='last')]
    df_pred['Spread_x'] = np.where(df_pred.Spread_x.isnull(), df_pred.Spread_y, df_pred.Spread_x)
    df_pred['Total_x'] = np.where(df_pred.Total_x.isnull(), df_pred.Total_y, df_pred.Total_x)
    df_pred = df_pred.rename(columns={"Spread_x": "Spread", "Total_x": "Total"}).drop(['Spread_y', 'Total_y'], axis=1)
    df_pred_mins = setup_df_mins(con, df_pred)
    df_pred_mins = df_pred_mins.drop(['Date', 'MP'], axis=1)
    df_pred['MP'] = mins_model.predict(df_pred_mins)

    # Predict Stat
    df_pred = setup_df_main(df_pred, tgt_stat)
    feature_cols = [col for col in df_pred.columns if col not in ['Date', tgt_stat]]
    df_pred = df_pred[df_pred.Date == now][feature_cols]
    df_pred[f"{tgt_stat}_proj"] = stat_model.predict(df_pred)

    # Setup results
    df_pred['Team'] = team_encoder.inverse_transform(df_pred["Team"])
    df_lines['Team'] = team_encoder.inverse_transform(df_lines["Team"])
    df_pred['Opp'] = team_encoder.inverse_transform(df_pred["Opp"])
    df_pred['Player'] = player_encoder.inverse_transform(df_pred["Player"])
    df_pred['Pos'] = position_encoder.inverse_transform(df_pred["Pos"])

    df_lines = df_lines[df_lines.Date == now][['Team', 'Player', f'{tgt_stat}_line']]
    df_pred = df_pred.merge(df_lines, on=['Team', 'Player'])
    df_pred = df_pred[~(df_pred[f'{tgt_stat}_line'].isnull())]
    df_pred['Diff'] = abs((df_pred[f'{tgt_stat}_line'] - df_pred[f'{tgt_stat}_proj']))
    df_pred['Diff2'] = abs((df_pred['MP'] - df_pred['MP_L5_avg']))
    df_pred = df_pred.sort_values('Diff', ascending=False).drop(['Diff', 'Diff2'], axis=1)
    
#     if df_pred.shape[0] >= 50:
#         print(df_pred.shape[0], 'rows')
#         for tm in df_pred.Team.unique():
#             display(df_pred[df_pred.Team == tm])
#     else:
#         display(df_pred)

    tds_picks = df_pred[['Team', 'Player', 'Pos', 'Opp', 'MP', 'MP_L5_avg', 'Spread', f'{tgt_stat}_line', f'{tgt_stat}_proj']]
    if tds_picks.shape[0] >= 50:
        print(tds_picks.shape[0], 'rows')
        for tm in tds_picks.Team.unique():
            display(tds_picks[tds_picks.Team == tm])
    else:
        display(tds_picks)
    tds_picks.insert(0, 'Date', pd.to_datetime(now))
    partition_save_df(tds_picks, f"../tables/2025/gmday_preds_{tgt_stat}.csv")

In [36]:
try: 
    generate_predictions('PTS')
except Exception as e:
    email('PTS', e)
    raise Exception(e)

130 rows


,Team,Player,Pos,Opp,MP,MP_L5_avg,Spread,PTS_line,PTS_proj
48,OKC,Aaron Wiggins,SG,MEM,16.282791,22.786942,-5.5,16.5,5.832615
36,OKC,Jalen Williams,SG,MEM,29.372309,29.769271,-5.5,24.5,14.630145
49,OKC,Isaiah Joe,SG,MEM,12.407850,16.739413,-5.5,12.5,3.887738
44,OKC,Ajay Mitchell,SG,MEM,23.966129,24.627837,-5.5,18.5,10.360072
99,OKC,Kenrich Williams,PF,MEM,13.191993,11.893158,-5.5,8.5,4.554811
75,OKC,Luguentz Dort,SF,MEM,22.774666,25.101059,-5.5,9.5,6.957831
54,OKC,Alex Caruso,SG,MEM,17.105499,17.952342,-5.5,6.5,4.582893


,Team,Player,Pos,Opp,MP,MP_L5_avg,Spread,PTS_line,PTS_proj
111,NYK,Karl-Anthony Towns,C,PHO,24.906101,25.506861,-1.5,20.5,12.143285
130,NYK,Mitchell Robinson,C,PHO,18.704988,21.125696,-1.5,4.5,2.444954
70,NYK,Mikal Bridges,SF,PHO,33.998806,34.288228,-1.5,14.5,12.824979
45,NYK,Jordan Clarkson,SG,PHO,17.571333,18.397832,-1.5,8.5,6.840978
87,NYK,OG Anunoby,PF,PHO,32.998955,33.326977,-1.5,15.5,14.540997
2,NYK,Jalen Brunson,PG,PHO,34.488003,34.598193,-1.5,27.5,27.834396
31,NYK,Miles McBride,SG,PHO,25.036356,26.853292,-1.5,12.5,12.803085


,Team,Player,Pos,Opp,MP,MP_L5_avg,Spread,PTS_line,PTS_proj
81,NOP,Zion Williamson,PF,WAS,26.190580,30.242472,-2.5,26.5,18.748280
10,NOP,Jordan Poole,PG,WAS,24.306465,26.201779,-2.5,17.5,11.415619
73,NOP,Trey Murphy III,SF,WAS,34.599148,33.789108,-2.5,20.5,25.588131
12,NOP,Jeremiah Fears,PG,WAS,22.140280,26.618625,-2.5,14.5,10.685475
53,NOP,Micah Peavy,SG,WAS,12.137867,16.282591,-2.5,6.5,3.937220
107,NOP,Derik Queen,C,WAS,24.213564,27.479860,-2.5,13.5,10.948198
125,NOP,Yves Missi,C,WAS,14.281077,19.471458,-2.5,5.5,5.122393


,Team,Player,Pos,Opp,MP,MP_L5_avg,Spread,PTS_line,PTS_proj
33,PHI,VJ Edgecombe,SG,ORL,36.547401,38.946546,-3.5,14.5,21.634254
109,PHI,Joel Embiid,C,ORL,31.668819,34.216103,-3.5,26.5,23.899687
0,PHI,Tyrese Maxey,PG,ORL,37.673630,40.517524,-3.5,27.5,30.030396
42,PHI,Quentin Grimes,SG,ORL,27.605007,34.435918,-3.5,9.5,11.930935
68,PHI,Kelly Oubre Jr.,SF,ORL,24.957644,27.812378,-3.5,9.5,10.921396
93,PHI,Paul George,PF,ORL,32.482571,33.439860,-3.5,15.5,14.642212


...

,Team,Player,Pos,Opp,MP,MP_L5_avg,Spread,PTS_line,PTS_proj
118,SAC,Maxime Raynaud,C,GSW,28.038351,31.580550,14.5,12.5,9.479255
84,SAC,DeMar DeRozan,PF,GSW,32.025387,31.659554,14.5,18.5,15.810113
7,SAC,Russell Westbrook,PG,GSW,26.969072,25.735043,14.5,14.5,16.793526
27,SAC,Zach LaVine,SG,GSW,31.792248,27.303308,14.5,20.5,18.301670
127,SAC,Precious Achiuwa,C,GSW,17.493502,20.054083,14.5,6.5,4.660679
8,SAC,Dennis Schroder,PG,GSW,22.092638,23.029098,14.5,11.5,12.641767
55,SAC,Nique Clifford,SG,GSW,19.586998,21.631104,14.5,5.5,4.683556


,Team,Player,Pos,Opp,MP,MP_L5_avg,Spread,PTS_line,PTS_proj
63,BOS,Jaylen Brown,SF,TOR,28.841700,35.753089,-10.5,28.5,25.578857
95,BOS,Sam Hauser,PF,TOR,20.053724,21.698784,-10.5,8.5,6.349222
15,BOS,Payton Pritchard,PG,TOR,32.644558,33.391291,-10.5,16.5,14.845963
28,BOS,Derrick White,SG,TOR,33.886139,36.110932,-10.5,18.5,16.984554
119,BOS,Neemias Queta,C,TOR,21.403513,24.762043,-10.5,9.5,8.223865
40,BOS,Anfernee Simons,SG,TOR,21.173712,25.748994,-10.5,13.5,12.864161
126,BOS,Luka Garza,C,TOR,17.353596,19.636189,-10.5,8.5,8.747617


,Team,Player,Pos,Opp,MP,MP_L5_avg,Spread,PTS_line,PTS_proj
41,MEM,Cam Spencer,SG,OKC,26.512283,28.984570,5.5,13.5,10.618794
116,MEM,Jock Landale,C,OKC,18.285673,22.278914,5.5,11.5,8.987701
106,MEM,Jaren Jackson Jr.,C,OKC,30.995518,34.046466,5.5,19.5,17.018202
97,MEM,Santi Aldama,PF,OKC,28.193518,30.287529,5.5,13.5,11.430147
38,MEM,Jaylen Wells,SG,OKC,26.878424,29.750980,5.5,13.5,12.071069
30,MEM,Cedric Coward,SG,OKC,20.253057,22.212802,5.5,13.5,14.144866


,Team,Player,Pos,Opp,MP,MP_L5_avg,Spread,PTS_line,PTS_proj
60,BRK,Michael Porter Jr.,SF,LAC,32.552326,33.195454,4.5,24.5,25.897739
121,BRK,Day'Ron Sharpe,C,LAC,19.012230,23.326320,4.5,6.5,7.755212
77,BRK,Ziaire Williams,SF,LAC,14.843341,19.733285,4.5,6.5,6.922568
98,BRK,Noah Clowney,PF,LAC,26.491991,28.311338,4.5,11.5,11.719281


../tables/2025/gmday_preds_PTS.csv saved!


In [37]:
try: 
    generate_predictions('PRA')
except Exception as e:
    email('PRA', e)
    raise Exception(e)

98 rows


,Team,Player,Pos,Opp,MP,MP_L5_avg,Spread,PRA_line,PRA_proj
48,OKC,Aaron Wiggins,SG,MEM,16.282791,22.786942,-5.5,24.5,8.805058
36,OKC,Jalen Williams,SG,MEM,29.372309,29.769271,-5.5,37.5,23.844677
44,OKC,Ajay Mitchell,SG,MEM,23.966129,24.627837,-5.5,28.5,16.845406
54,OKC,Alex Caruso,SG,MEM,17.105499,17.952342,-5.5,12.5,7.555272
75,OKC,Luguentz Dort,SF,MEM,22.774666,25.101059,-5.5,15.5,11.246674


,Team,Player,Pos,Opp,MP,MP_L5_avg,Spread,PRA_line,PRA_proj
81,NOP,Zion Williamson,PF,WAS,26.190580,30.242472,-2.5,37.5,23.937189
10,NOP,Jordan Poole,PG,WAS,24.306465,26.201779,-2.5,23.5,16.119759
12,NOP,Jeremiah Fears,PG,WAS,22.140280,26.618625,-2.5,21.5,15.788011
107,NOP,Derik Queen,C,WAS,24.213564,27.479860,-2.5,27.5,23.520824
73,NOP,Trey Murphy III,SF,WAS,34.599148,33.789108,-2.5,30.5,31.993673


,Team,Player,Pos,Opp,MP,MP_L5_avg,Spread,PRA_line,PRA_proj
80,MIL,Giannis Antetokounmpo,PF,LAL,23.464924,28.497827,3.5,47.5,37.374504
1,MIL,Kevin Porter Jr.,PG,LAL,36.677925,38.750643,3.5,30.5,33.625320
4,MIL,Ryan Rollins,PG,LAL,30.194422,34.506348,3.5,24.5,24.544058


,Team,Player,Pos,Opp,MP,MP_L5_avg,Spread,PRA_line,PRA_proj
111,NYK,Karl-Anthony Towns,C,PHO,24.906101,25.506861,-1.5,34.5,24.943914
2,NYK,Jalen Brunson,PG,PHO,34.488003,34.598193,-1.5,37.5,31.840801
70,NYK,Mikal Bridges,SF,PHO,33.998806,34.288228,-1.5,23.5,20.278154
31,NYK,Miles McBride,SG,PHO,25.036356,26.853292,-1.5,18.5,15.531443
87,NYK,OG Anunoby,PF,PHO,32.998955,33.326977,-1.5,22.5,20.010517
45,NYK,Jordan Clarkson,SG,PHO,17.571333,18.397832,-1.5,11.5,9.966813


...

,Team,Player,Pos,Opp,MP,MP_L5_avg,Spread,PRA_line,PRA_proj
66,LAL,LeBron James,SF,MIL,31.296171,32.772519,-3.5,35.5,38.546474
47,LAL,Marcus Smart,SG,MIL,28.039499,31.625469,-3.5,15.5,17.004002
5,LAL,Luka Doncic,PG,MIL,34.551426,37.403849,-3.5,52.5,51.814037


,Team,Player,Pos,Opp,MP,MP_L5_avg,Spread,PRA_line,PRA_proj
13,ORL,Anthony Black,PG,PHI,30.665329,31.098146,3.5,26.5,24.362577
82,ORL,Paolo Banchero,PF,PHI,33.707272,35.879836,3.5,37.5,35.695892
24,ORL,Desmond Bane,SG,PHI,32.415100,34.691565,3.5,28.5,26.765919


,Team,Player,Pos,Opp,MP,MP_L5_avg,Spread,PRA_line,PRA_proj
98,BRK,Noah Clowney,PF,LAC,26.491991,28.311338,4.5,17.5,16.531845
121,BRK,Day'Ron Sharpe,C,LAC,19.012230,23.326320,4.5,16.5,16.306953
60,BRK,Michael Porter Jr.,SF,LAC,32.552326,33.195454,4.5,35.5,35.412312


,Team,Player,Pos,Opp,MP,MP_L5_avg,Spread,PRA_line,PRA_proj
27,SAC,Zach LaVine,SG,GSW,31.792248,27.303308,14.5,26.5,25.651878
7,SAC,Russell Westbrook,PG,GSW,26.969072,25.735043,14.5,27.5,27.970993
8,SAC,Dennis Schroder,PG,GSW,22.092638,23.029098,14.5,19.5,19.340328
84,SAC,DeMar DeRozan,PF,GSW,32.025387,31.659554,14.5,25.5,25.366224


../tables/2025/gmday_preds_PRA.csv saved!
